In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import re 

In [2]:
data = pd.read_csv("data.csv")

In [3]:
data.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...


In [4]:
data.shape

(7787, 12)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


In [6]:
data.isnull().sum()

show_id            0
type               0
title              0
director        2389
cast             718
country          507
date_added        10
release_year       0
rating             7
duration           0
listed_in          0
description        0
dtype: int64

### **IDEA** : MAKE A MODEL THAT SUGGESTS SHOWS BASED ON MINIMUM DESCRIPTION, SHOW TYPE, COUNTRY/LANGUAGE, RELEASE YEAR AND TITLE

In [7]:
df = data.copy()

In [8]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [9]:
for  x in df.columns:
    print("-"*100)

    if len(df[x].unique()) < 20:
        print(f"{x} : {df[x].unique()}")
    else:
        print(f"{x} : {len(df[x].unique())} values")


----------------------------------------------------------------------------------------------------
show_id : 7787 values
----------------------------------------------------------------------------------------------------
type : ['TV Show' 'Movie']
----------------------------------------------------------------------------------------------------
title : 7787 values
----------------------------------------------------------------------------------------------------
director : 4050 values
----------------------------------------------------------------------------------------------------
cast : 6832 values
----------------------------------------------------------------------------------------------------
country : 682 values
----------------------------------------------------------------------------------------------------
date_added : 1566 values
----------------------------------------------------------------------------------------------------
release_year : 73 values
----------

In [10]:
df[df.director.isnull()].head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
11,s12,TV Show,1983,NaN,"Robert Więckiewicz, Maciej Musiał, Michalina O...","Poland, United States","November 30, 2018",2018,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Dramas","In this dark alt-history thriller, a naïve law..."
16,s17,TV Show,Feb-09,NaN,"Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-...",NaN,"March 20, 2019",2018,TV-14,1 Season,"International TV Shows, TV Dramas","As a psychology professor faces Alzheimer's, h..."
19,s20,Movie,'89,NaN,"Lee Dixon, Ian Wright, Paul Merson",United Kingdom,"May 16, 2018",2017,TV-PG,87 min,Sports Movies,"Mixing old footage with interviews, this is th..."
24,s25,TV Show,​SAINT SEIYA: Knights of the Zodiac,NaN,"Bryson Baugus, Emily Neves, Blake Shepard, Pat...",Japan,"January 23, 2020",2020,TV-14,2 Seasons,"Anime Series, International TV Shows",Seiya and the Knights of the Zodiac rise again...


### Some guideline to follow by imdb to scrap and use data from their site.

![Screenshot (268).png](<attachment:Screenshot (268).png>)

### many values are missing and the information is only available on internet. 
### I will scrap the missing data from IMDb using bs4.

In [11]:
from bs4 import BeautifulSoup
import requests

In [12]:
reqs = requests.get(r"https://www.imdb.com/chart/top/")

In [13]:
site = BeautifulSoup(reqs.text, 'html.parser')

In [14]:
print(site.text)


403 Forbidden

403 Forbidden





### Its forbidden, lets try using selenium web driver.

In [103]:
import unicodedata 
def valid_content(df, df_index, result_search, title, path, not_found = None): 

    result_search = result_search.lower()
   
    # extract data for the given title from dataframe
    aux = df.iloc[df_index]
    # title from dataframe
    title_Df = aux.title

    # cleaning the invisible charcters
    title_Df = ''.join(char for char in title_Df if unicodedata.category(char)[0] != 'C')

    #print(title_Df, title)
    # match the titles
    title_bool = (title.lower().strip() == title_Df.lower().strip())
    # if title doesnt match return False
    if not title_bool:
        #print(f"Not title bool match        input : {title_Df}          Search : {title}")
        return False

    year = str(aux.release_year) in result_search

    # get cast and lower the alphabet so that it can be used for conditional checking
    cast_ = aux['cast']

    # check nans
    if type(cast_) != str:
        cast_bool = False
        
    else:
        cast_ = list(map(str.lower, (aux['cast']).split(',')))
        for c in cast_:
            if c in result_search:
                cast_bool = True
                break
            else:
                cast_bool = False
                break

    # check whether the content is TV show or Movie
    type_TV = "tv" in result_search 
    
    if type_TV == True and aux.type == 'TV Show':
        type_bool = True
    elif type_TV == False and aux.type == 'Movie':
        type_bool = True
    else:
        type_bool = False
    

    # make list of results, remove if nans 
    movie_info = [year, cast_bool, type_bool]
    for m in movie_info:
        # check for nan and deletes it
        if type(m) != str:
            movie_info.remove(m)
    #print(movie_info)
    # for the case where we have minimum info in search result; titles match but because of not having 
    # required information it gives false, to solve this case we use movie title bool to that and if it is 
    # a match then return True    [This might just take any title that matches but cant do much for this case]'
    if not_found:
        if title_bool:
            #print(f"match in less info case        input : {title_Df}          Search : {title}")
            return True
        else:
            #print(f"Not match in less info case        input : {title_Df}          Search : {title}")
            return False

    elif not_found == None:
        #print(f"Case of 1st iteration        input : {title_Df}          Search : {title}")
        return any(movie_info)






# In Case of finding not valid content ( finding similar titled content then this will iterate through search results
# and find valid content and if not found yhen return False)
def xpath_to_valid_content(df, df_index):
 
    search_item_index = 1
    not_found = None
    while search_item_index < 6:
        try:
            # path to search results
            xpath_ = f"//*[@id='__next']/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[{search_item_index}]"
                    #  '//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[1]/div[2]/div/a'
            search_result = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, xpath_)))            
            search_result = search_result.text

            # get the title from website
            title = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, xpath_+'/div[2]/div/a'))).text
            # give a bool that says whether the search result meets our content data
            # this is to avoid mix up between same movie titles or series
            # the function valid_content check other features of the content
            # check validity of current content
            valid_cont = valid_content(df, df_index, search_result, title, xpath_, not_found)

            # if valid content then return the path
            if valid_cont:
                return xpath_

            if search_item_index == 5 and not_found == None:
                not_found = True
                search_item_index = 1
                continue

            search_item_index +=1


        except TimeoutException:
            print(f"Not Found : {df.iloc[df_index].title}")
            return False

        except ElementClickInterceptedException:
            new_series = df[df.cast.isnull()].iloc[df_index:]
            new_series = new_series.title
            print(f"Click Interception : {df.iloc[df_index].title}" )
            scrap_and_fill(df, new_series, 'cast')

    # after all these iterations if not valid content found then return path as false
    if not valid_cont:
        print(f"Not Found : {df.iloc[df_index].title}")
        return False
    


def extract(title, elem):
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #try:
    # for cast 
    if elem == 'Cast':
        # loop to find correct section for the cast informatiom
        for j in range(2, 6):
            path = f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[{j}]/div[2]/div[2]/'
            try:
                cast = []
                # loop to extract 5 cast names
                for i in range(1, 6):
                    # find the element that shows that the next content is cast or else
                    elem_path = path + f"div[{i}]/div[2]/a"
                    elem_text = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, elem_path)))
                    elem_text = elem_text.text
                    cast.append(elem_text)

                final_cast = ", ".join(cast)
                print(f"{title} : {final_cast}")
                return final_cast
                
            except TimeoutException:
                # if we found the cast and other elements couldnot be found because they dont exist and ebcasue of that
                # we got timeout exception return the cast and break the loop 
                if len(cast) > 0:
                    final_cast = ", ".join(cast)
                    print(f"Cast Time out {j}")
                    print(f"{title} : {final_cast}")
                    return final_cast
                # if len(cast) is 0 then continue to next loop
                continue
    # for director
    else:
        path = '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/'
        # director information is sure to be found after section in or after sectionn 2
        for i in range(2, 6):
            try:
                # take path directly to our content and extract and return the text
                elem_text = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, path + f'section[{i}]/ul/li[1]/div')))
                elem_text = elem_text.text
                director = ",".join(elem_text.split())
                print(f"{title} : {director}")
                return director

            except TimeoutException:
                continue
        

def fill_and_click(i, series):
    try:
        search_box = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@id='suggestion-search']")))
        search_box.send_keys(series[i])
            
        # Find and click the search button
        search_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='suggestion-search-button']")))
        search_button.click()

    except TimeoutException:
        print("Timed out on search, going to home page. Restarting the previous search")
        driver.get("https://www.imdb.com/")
        search_box = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@id='suggestion-search']")))
        search_box.send_keys(series[i])
            
        # Find and click the search button
        search_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='suggestion-search-button']")))
        search_button.click()

def scrap_and_fill(df, series, feature):
    
    
    #driver.execute_script("window.scrollTo(0,0);")
    driver.get("https://www.imdb.com/")

    for df_index in series.index:

        # Find and fill the title in search box
        fill_and_click(df_index, series)

        """on the next page / search result page"""
        # path to valid search result
        path = xpath_to_valid_content(df, df_index)

        # if not content does not match, find the matching content from other results
        # possibility of not getting the exact content info from IMDb. then skip this content 
        if path == False: continue

        try:
            # if have a valid path then go ahead and click
            next_page = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, path)))
            next_page.click()

        # Incase when element click is intercepted by pop-ups
        except ElementClickInterceptedException:
            new_series = df[df.cast.isnull()].iloc[df_index:]
            new_series = new_series.title
            scrap_and_fill(df, new_series, feature)
            

        
        """on the page of clicked title"""

        # conditional filling
        if feature.lower() == 'director':
            #driver.execute_script("window.scrollTo(0, 1000);")
            dir_ = extract(df.iloc[df_index].title, 'Director')
            if dir_ == False:
                continue
            else:
                df.loc[df_index, "director"] = dir_
        else:
            cast_ = extract(df.iloc[df_index].title, 'Cast')
            if cast_ == False: 
                continue
            else:
                df.loc[df_index, "cast"] = cast_

                



In [16]:
df_exp = df.copy()

In [12]:
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException 
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import ElementClickInterceptedException

In [18]:
# Initialize Chrome WebDriver with headless mode
driver = webdriver.Chrome()

no_cast_titles = df[df['cast'].isnull()].title
scrap_and_fill(df_exp, no_cast_titles, "cast")
driver.quit()

Stefano Accorsi, Miriam Leone, Guido Caprino, Paolo Mazzarelli, Antonio Gerardi
Cast Time out 3
Khalifah Ali Al-Akili, Mike Healey, Ali Kareem, Saeed 'Shariff' Torres
Cast Time out 4
Jennifer Julian
Kevin Allocca, Phaedra Barratt, Amanda Brennan, Mike Bridavsky, Pamela Chen
Cast Time out 2
Inez Dickens
Not Found : show_id                                                       s53
type                                                        Movie
title                                        100 Days Of Solitude
director                                                      NaN
cast                                                          NaN
country                                                     Spain
date_added                                           July 6, 2018
release_year                                                 2018
rating                                                      TV-MA
duration                                                   93 min
listed_in               

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (550, 759)
  (Session info: chrome=122.0.6261.95)
Stacktrace:
	GetHandleVerifier [0x00007FF77460AD22+56930]
	(No symbol) [0x00007FF77457F622]
	(No symbol) [0x00007FF7744342E5]
	(No symbol) [0x00007FF774480A80]
	(No symbol) [0x00007FF77447E8AB]
	(No symbol) [0x00007FF77447C0B4]
	(No symbol) [0x00007FF77447AE45]
	(No symbol) [0x00007FF77446F798]
	(No symbol) [0x00007FF77449BC9A]
	(No symbol) [0x00007FF77446F09A]
	(No symbol) [0x00007FF77449BEB0]
	(No symbol) [0x00007FF7744B81E2]
	(No symbol) [0x00007FF77449BA43]
	(No symbol) [0x00007FF77446D438]
	(No symbol) [0x00007FF77446E4D1]
	GetHandleVerifier [0x00007FF774986AAD+3709933]
	GetHandleVerifier [0x00007FF7749DFFED+4075821]
	GetHandleVerifier [0x00007FF7749D817F+4043455]
	GetHandleVerifier [0x00007FF7746A9756+706710]
	(No symbol) [0x00007FF77458B8FF]
	(No symbol) [0x00007FF774586AE4]
	(No symbol) [0x00007FF774586C3C]
	(No symbol) [0x000

In [19]:
df_exp.isnull().sum()

show_id            0
type               0
title              0
director        2389
cast             664
country          507
date_added        10
release_year       0
rating             7
duration           0
listed_in          0
description        0
dtype: int64

### After filling 211 values, because of pop up banner the exception created. We will continue this further scraping, from df_exp dataframe.
### We also have to ignore the titles that weren't found during the search.

In [26]:
# So there were 718 nulls in cast. After above code execution we still have 507 values null. 211 values are filled
# current title is 'blown away'
df_exp[df_exp.title == 'Blown Away']

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
994,s995,TV Show,Blown Away,NaN,NaN,Canada,"July 12, 2019",2019,TV-14,1 Season,"International TV Shows, Reality TV",Ten master artists turn up the heat in glassbl...


In [12]:
df_exp = pd.read_csv("incomplete_scrapped_df(only cast).csv")

In [13]:
# we got the index
df_cont = df[df.cast.isnull()]
df_cont = df_cont[df_cont.index >= 994]
remaining_titles = df_cont.title 

In [27]:
del driver
# Initialize Chrome WebDriver with headless mode
driver = webdriver.Chrome()
scrap_and_fill(df_exp, remaining_titles, "cast")
driver.quit()

Jeff Bridges, Tommy Lee Jones, Suzy Amis, Lloyd Bridges, Forest Whitaker
Marina Abramovic, Katherine Arnold, Leon Black, Amy Cappellazzo, Simon de Pury
Cherprang Areekul, Nayika Srinian, Rina Izuta, Napaphat Worraphuttanon, Kunjiranut Intarasin
Gerry Adams, Michael Alison, Humphrey Atkins, Aram Bakshian, Mario Biaggi
Juan Pablo Varsky, Carlos Tevez, Guillermo Barros Schelotto, Boca Juniors, Wanchope Ábila
Cast Time out 4
Jeff Cole
Cast Time out 3
Walaa Eltiti, Yazeed Faruja
Cast Time out 3
Tia Alkerdi, Javier Castell
Gustavo Santaolalla, Rubén Albarrán, Javier Batiz, Humberto Carderon, Camilo Lara
Charlie Booty, Lily James, Tim Roth, Eloise Laurence, Lukas Fernandes-Pendse
Jaye Griffiths, Jesse Birdsall, Craig McLachlan, Jan Harvey, Paula Hunt
Brad Renfro, Nick Stahl, Bijou Phillips, Rachel Miner, Michael Pitt
Not found : 
Jett Adore, Russell Bruner, Angelique Devil, Ray Gunn, Babs Jamboree
Not found : 
Not Found : Unnamed: 0                                                   1177
show_

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF73614AD22+56930]
	(No symbol) [0x00007FF7360BF622]
	(No symbol) [0x00007FF735F742E5]
	(No symbol) [0x00007FF735FB98ED]
	(No symbol) [0x00007FF735FB9A2C]
	(No symbol) [0x00007FF735FFA967]
	(No symbol) [0x00007FF735FDBCDF]
	(No symbol) [0x00007FF735FF81E2]
	(No symbol) [0x00007FF735FDBA43]
	(No symbol) [0x00007FF735FAD438]
	(No symbol) [0x00007FF735FAE4D1]
	GetHandleVerifier [0x00007FF7364C6AAD+3709933]
	GetHandleVerifier [0x00007FF73651FFED+4075821]
	GetHandleVerifier [0x00007FF73651817F+4043455]
	GetHandleVerifier [0x00007FF7361E9756+706710]
	(No symbol) [0x00007FF7360CB8FF]
	(No symbol) [0x00007FF7360C6AE4]
	(No symbol) [0x00007FF7360C6C3C]
	(No symbol) [0x00007FF7360B68F4]
	BaseThreadInitThunk [0x00007FFEBAB954E0+16]
	RtlUserThreadStart [0x00007FFEBBE0485B+43]


In [30]:
df_cont = df[df.cast.isnull()]
df_cont = df_cont[df_cont.index >= 5056]
remaining_titles = df_cont.title 

In [36]:
len(remaining_titles)

86

In [32]:
del driver
# Initialize Chrome WebDriver with headless mode
driver = webdriver.Chrome()
scrap_and_fill(df_exp, remaining_titles, "cast")
driver.quit()

Not Found : Unnamed: 0                                                   5056
show_id                                                     s5057
type                                                        Movie
title           Radical: the Controversial Saga of Dada Figuei...
director                                         Raphael Erichsen
cast                                                          NaN
country                                                    Brazil
date_added                                         April 12, 2017
release_year                                                 2013
rating                                                      TV-MA
duration                                                   71 min
listed_in       Documentaries, International Movies, Sports Mo...
description     This film chronicles the career of surfer Dadá...
Name: 5056, dtype: object
Chris Strompolos, Eric Zala, Jayson Lamb, John Rhys-Davies, Chris Gore
Ed Sheehan, Bobby Corrigan, Rick 

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF73614AD22+56930]
	(No symbol) [0x00007FF7360BF622]
	(No symbol) [0x00007FF735F742E5]
	(No symbol) [0x00007FF735FB98ED]
	(No symbol) [0x00007FF735FB9A2C]
	(No symbol) [0x00007FF735FFA967]
	(No symbol) [0x00007FF735FDBCDF]
	(No symbol) [0x00007FF735FF81E2]
	(No symbol) [0x00007FF735FDBA43]
	(No symbol) [0x00007FF735FAD438]
	(No symbol) [0x00007FF735FAE4D1]
	GetHandleVerifier [0x00007FF7364C6AAD+3709933]
	GetHandleVerifier [0x00007FF73651FFED+4075821]
	GetHandleVerifier [0x00007FF73651817F+4043455]
	GetHandleVerifier [0x00007FF7361E9756+706710]
	(No symbol) [0x00007FF7360CB8FF]
	(No symbol) [0x00007FF7360C6AE4]
	(No symbol) [0x00007FF7360C6C3C]
	(No symbol) [0x00007FF7360B68F4]
	BaseThreadInitThunk [0x00007FFEBAB954E0+16]
	RtlUserThreadStart [0x00007FFEBBE0485B+43]


In [34]:
df[df.title == 'The Silence of Others']

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
6871,s6872,Movie,The Silence of Others,"Almudena Carracedo, Robert Bahar",NaN,"Spain, France, Canada, United States","July 1, 2020",2019,TV-14,96 min,"Documentaries, International Movies","Filmed over six years, this documentary captur..."


In [35]:
df_cont = df[df.cast.isnull()]
df_cont = df_cont[df_cont.index >= 6871]
remaining_titles = df_cont.title 

In [37]:
del driver
# Initialize Chrome WebDriver with headless mode
driver = webdriver.Chrome()
scrap_and_fill(df_exp, remaining_titles, "cast")
driver.quit()

Cast Time out 4
Max Park, Schwan Park, Philipp Weyer, Feliks Zemdegs
Muammar Gaddafi, Henry Kissinger, Moshe Dayan, Nikita Khrushchev, Thomas Meadmore
Colin Firth, Toni Collette, Michael Stuhlbarg, Patrick Schwarzenegger, Sophie Turner
Tim Scully, Nick Sand, Alice Einhorn, Billy Hitchcock, Timothy Leary
Cast Time out 5
Lin Gallagher
Allan Abramson, Keith Arnold, Terence Benson, Michael Chen, James Davies
Juan Manuel Fraire Escobedo, Alejandro Fraire, Blanca Escobedo, Patricia González, Ruth Fierro
Donald Ian Black, Kirk Bozigian, Mark Bellomo, David Vonner, James Eatock
Not found : 
William Guirola, Wyatt Hinz, Alfredo Tavares, Adam Saunders, Rodrigo Alfonso
Aidan Gallagher, Elliot Page, Tom Hopper, David Castañeda, Emmy Raver-Lampman
Not Found : Unnamed: 0                                                   6980
show_id                                                     s6981
type                                                      TV Show
title           The Vietnam War: A Film by Ke

In [40]:
df_exp.cast.isnull().sum()

237

In [ ]:
df_exp.to_csv("cast_filled.csv")

In [13]:
df_exp = pd.read_csv("cast_filled.csv")

### There are still null values, that is due to not found results in scrapping.

### Now lets do the same for missing directors.

In [75]:
y = """1983
2018tv series
robert wieckiewicz, maciej musial"""
print(str(2018) in y)

True


In [22]:
title_ = df[df.director.isnull()]['title']

In [104]:
driver = webdriver.Chrome()
scrap_and_fill(df_exp, title_, "director")
driver.quit()

Not Found : 3%
1983 : Joshua,Long
Not Found : Feb-09
Not Found : '89
#blackAF : Kenya,Barris
Not Found : แผนร้ายนายเจ้าเล่ห์
Not Found : Şubat
Not Found : 100 Days Of Solitude
Not Found : 100 Humans
100 Things to do Before High School : Scott,Fellows
13 Reasons Why : Brian,Yorkey
20 Minutes : Timothy,A.,Chey
Not Found : 21 Sarfarosh: Saragarhi 1897
21 Thunder : Riley,AdamsKenneth,HirschAdrian,Wills
Not Found : 28 Moons
3Below: Tales of Arcadia : Guillermo,del,Toro
44 Cats : Iginio,Straffi
45 rpm : Ramón,CamposGema,R.,Neira
Not Found : 72 Dangerous Animals: Asia
Not Found : A Boy Name Flora A
Not Found : A Christmas Catch
A Good Wife : Mirjana,Karanovic
Not Found : A House of Blocks
A Little Help with Carol Burnett : Carol,Burnett
A Man Called God : Christopher,St.,JohnKristoff,St.,John
A new Capitalism : Henry,Grazinoli
A Plastic Ocean : Craig,Leeson(attached)
Not Found : A Taiwanese Tale of Two Cities
A Very Secret Service : Jean-François,HalinClaire,LemaréchalJean-André,Yerlès
Not Fo

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF70D83AD22+56930]
	(No symbol) [0x00007FF70D7AF622]
	(No symbol) [0x00007FF70D6642E5]
	(No symbol) [0x00007FF70D6A98ED]
	(No symbol) [0x00007FF70D6A9A2C]
	(No symbol) [0x00007FF70D6EA967]
	(No symbol) [0x00007FF70D6CBCDF]
	(No symbol) [0x00007FF70D6E81E2]
	(No symbol) [0x00007FF70D6CBA43]
	(No symbol) [0x00007FF70D69D438]
	(No symbol) [0x00007FF70D69E4D1]
	GetHandleVerifier [0x00007FF70DBB6AAD+3709933]
	GetHandleVerifier [0x00007FF70DC0FFED+4075821]
	GetHandleVerifier [0x00007FF70DC0817F+4043455]
	GetHandleVerifier [0x00007FF70D8D9756+706710]
	(No symbol) [0x00007FF70D7BB8FF]
	(No symbol) [0x00007FF70D7B6AE4]
	(No symbol) [0x00007FF70D7B6C3C]
	(No symbol) [0x00007FF70D7A68F4]
	BaseThreadInitThunk [0x00007FFEBAB954E0+16]
	RtlUserThreadStart [0x00007FFEBBE0485B+43]


### Timed out because the site didnt load

In [107]:
df_exp.to_csv("imputed.csv")